# PLease this time, before Anyone make any changes, please let me know, 
# Since Mam Has given us one more chance, I don't want to miss it
# thanks

## Import Statements

In [3]:
# used to parse the files
import xml.etree.ElementTree as xmltree

# for searching the folders
from os.path import isdir
from os.path import isfile
from os import mkdir
from os import listdir

# File Processing Functions

In [20]:
# this will directly dump the word_tag pairs from give file in the their corresponding word_tag files
# in sentence format

# so the format is 
# every line of dest file will be a sentence
# we will only consider word tags this time.
# and sentences will be stored as word_tag pairs

def parseFile_mk(source,dest) :
    
    # xmlfile is the file to be handled
    xmlfile = open (source, "r",encoding= "utf-8")
    # we get the text into a variable
    xmlfiletext = xmlfile.read();
    #close xmlfile
    xmlfile.close()
    
    # make am xml tree from that text
    tree = xmltree.fromstring(xmlfiletext)
    
    # make list of sentences
    slist = tree.iter('s')
    
    #open dest file with **overwriting** mode
    dfile = open (dest[:-4]+"_sentence.txt", "w+", encoding = "utf-8")
    
    
    # put the data in the sfile
    for sentence in slist :
        # declare a line variable to store the sentence
        line = ""
        # iterate over every variable in sentence
        for elem in sentence :
            #make sure we only process words for a sentence
            if elem.tag != 'w' :
                continue 
            # fetch pos tag of this elem
            pos = elem.get('c5')
            # if the child doesnot contains c5 then it is of no use for us
            if pos is None :
                continue
                
            # upper case all pos tags    
            pos = pos.upper()   
            
            # some where the authors were confused and they assigned two tags
            # in such scenarios we are deciding to go with first tag and ignore the second
            pos = pos[:3]
            
            # if I am here it  means pos has c5 tag available so we need to find the text of it.
            txt = elem.text
            if txt is None : 
                continue
            txt = txt.strip()
            # append in the sentence this word
            line =   line +      txt + "_"+ pos        +" "  # extra space added between terms 
                                                             # to just make more clear
        # for word loop finshes
        # write the sentence to the file
        dfile.write(line+"\n")	
    # for sentence loop finished 
    #close file
    dfile.close()

In [21]:
# function
# this will fetch all the files and generate corresponding files
# this takes source folder and 
def makeSentences_mk(source, dest) :

    #check if the source is a file
    if isfile(source):
        # if yes, then process the file 
        # if destination is not there , the function will create it, and if it is there, it will overrite it.
        parseFile_mk(source,dest)
    else:
        # so the source is a folder, we need to make corresponding folder in the dest 
        #check if dest has the folder
        if not isdir(dest):
            mkdir(dest)
        
        # now enumerate all the items in the source directory    
        for item in listdir(source):
            makeSentences_mk(source+'/'+item,dest+'/'+item)



# Please Uncomment & Run below Cell to Process Files, 
## I do processed my files so I have commented this to avoid repeats

In [22]:
# # this segment is used to make the files process all togethor in their own files

# source = "given_data"
# dest = "processed_data"
# makeSentences_mk(source, dest)

# Combine Function to Combine all the files

In [23]:
# Collector function, this will dump all sentences into the destination file
def dump_mk(source,dest):

    sfile = open(source, "r", encoding= "utf-8")
    dfile = open(dest, "a", encoding= "utf-8")

    # read entire sfile line by line, yes this is the key
    # Possible Error Point : in our Previous Implementation I was reading It all at same time, this was problem ,
        # whole file became a single sentence
    # and then dump that  into the dfile as itis.
    for line in sfile :
        dfile.write(line + "\n")

    sfile.close()
    dfile.close()

# this iterates and finds all the files and stores them into a single file 
# using dumping function
def combine_mk(source, dest) :

    #check if the source is a file
    if isfile(source):
        # if yes, then process the file 
        # if destination is not there , the function will create it, and if it is there, it will overrite it.
        dump_mk(source,dest)
    else:
        # so the source is a folder, 
        # now enumerate all the items in the source directory  
        # and take appropriate actions  
        for item in listdir(source):
            combine_mk(source+'/'+item,dest)



# Please Uncomment Below TWO Cells to Combine the Test & Train Data
## I did it thus I have left them commented

If you need anyhelp Sritam, just whatsapp me .. 

In [24]:
# # used to combine all the processed files for both train & test data sets 
# # together one to each other in their respective files

# # Combine Test Data
# s="processed_data/test_corpus"
# d = "processed_data/test_combine.txt"
# dfile = open(d,"w+", encoding="utf-8")
# dfile.close()

# combine_mk(s,d)

In [25]:
# # Combine Train Data
# s="processed_data/train_corpus"
# d = "processed_data/train_combine.txt"
# dfile = open(d,"w+", encoding="utf-8")
# dfile.close()

# combine_mk(s,d)

# Calculating Emission Probabilities

I saw many videos on youtube with many examples, now I think this is first step we should do. 
And if we can do this nicely, this will give us COnfidence as well :)

## What Does Emission Probability means?
Make this thing clear, so that implementation is not wrong.

Hidden states are tags 
And observations are words... 

So emission probability means, for word we calculate the chance that given tag, what is probability this word will occur 

Emission P of word w with tag t = Count of word_tag pairs with word = w and tag = t divided by count of word_tag pairs with tag equal to t no matter what word is....

### So far so good ... 

Obviously this makes things clear.

I will make a dictionary counting appearnace of every tag , this will give me count of word_tag pairs with tag t for every tag t

Or better make dictionary of dictionary
Dict with keys as tags, and second dictionary with keys as words ? how's this ... seems fine right... this will give us all the data we need to go 

Finally to save it I will pickle the struct, and will see if I need to do anything else.

In [ ]:
# the emission probabilty calculator functtion

def emission_mk():
    # obviosly source file is 
    data = open("processed_data/train_combine.txt", "r", encoding="utf-8")
    
    # make dictonary for tags as keys
    tag_dict = {}
    
    # now read every line one by one of this file
    for line in data :
        
        # split line into word_tag pairs with split
        pairs = line.split()
        
        # by pass empty line
        if len(pairs) < 1: 
            continue
        
        # iterate over pairs
        for pair in pairs :
            
        
        
        
        